# Višeklasna klasifikacija

Ukoliko je broj klasa u zadatku klasifikacije veći od dva govorimo o višeklasnoj klasifikaciji (engl. multiclass classification). Za nju važe ista pravila pripreme podataka, kreiranja modela i evaluacije koja smo uveli u priči o binarnoj klasifikaciji. Binarni algoritmi klasifikacije se mogu uvek prilagoditi ovom viešklasnom scenariju kroz omotače `OneVsAll` i `OneVsOne` biblioteke `scikit-learn`, a neki od njih i kroz specifični parametar `multi_class`.  

Pretpostavimo da treba da naučimo da razlikujemo n=3 klasa. Obeležimo ih redom sa C1, C2 i C3.

`OneVsAll` (ili `oneVsRest`) pristup podrazumva učenje n binarnih klasifikatora koji mogu da razlikuju jednu uočenu klasu od preostalih klasa. Na primer, potrebno je naučiti klasifikator koji razdvaja klasu C1 od klasa C2 i C3, zatim klasifikator koji razdvaja klasu C2 od klasa C1 i C3, i na kraju, klasifikator koji razdvaja klase C3 od klasa C1 i C2. Kada je potrebno klasifikovati novu instancu, pokreće se svaki od naučenih klasifikatora i nad dobijenim rezultatima se primenjuje princip najveće pouzdanosti: instanca se pridružuje klasi za koju je klasifikator najsigurniji.

<img src='assets/OneVsRest.png'>

`OneVsOne` pristup podrazumeva učenje $\frac{n(n-1)}{2}$ binarnih klasifikatora sposobnih da razlikuju po dve klase. U našem primer, potrebno je naučiti binarne klasifikatore koji razdvajaju klase C1 i C2, C2 i C3, kao i C1 i C3. Kada je potrebno klasifikovati novu instancu, pokreće se svaki od naučenih klasifikatora i nad dobijenim rezultatima se primenjuje princip većinskog glasanja: instanca se pridružuje klasi za koju najveći broj klasifikatora glasa.

<img src='assets/OneVsOne.jpg'>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
from sklearn import datasets
from sklearn import svm
from sklearn import multiclass
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
np.random.seed(7)

## Primer 1.

Ovaj primer približava rad sa višeklasnim klasifikatorima i omotačima `OneVsRest` i `OneVsOne`. 

## Primer 2.

Ovaj primer obrađuje problem identifikacije (prepoznavanja) jezika nekog teksta. Ova funkcionalnost je često integrisana u aplikacijama koje svakodnevno susrećemo. Na primer, prilikom pretraživanja informacija potrebno je odrediti jezik upita, a prilikom predlaganja korekcija u tekstu, jezik dokumenta. U primeru ćemo koristiti fragmente teksta preuzete sa Vikipedije i podršku `scikit-learn` biblioteke za njihovu dalju obradu. Klasifikator `linearni SVM` ćemo ovog puta videti u ulozi višeklasnog klasifikatora. 

Za prevlačenje fragmenata teksta sa Vikipedije koristićemo skipt `fetch_data.py` koji je preuzet sa [zvanične strane](https://github.com/scikit-learn/scikit-learn/blob/master/doc/tutorial/text_analytics/data/languages/fetch_data.py) biblioteke `scikit-learn`. Njime će se u radnom direktorijumu kreirati tri direktorijuma, `html`, `paragraphs` i `short_paragraphs` sa tekstovima razvrstanim po jezicima (ukupno njih 11). Mi ćemo u daljem radu koristiti samo `paragraphs` direktorijum. 

Prevlačimo paragrafe.

In [ ]:
!python scripts/fetch_data.py

In [20]:
languages_folder = 'paragraphs'

U zadatku prepoznavanja jezika su se dobro pokazali **n-grami karaktera** tj. sekvence nekoliko uzastopnih karaktera. Na primer, svi unigrami (1-grami) reči "abcde" su "a", "b", "c", "d", "e", svi bigrami (2-grami) iste reči su "ab", "bc", "cd", "de", dok su svi 3-grami "abc", "bcd", "cde".  Njih možemo lako generisati korišćenjem funkcija za ekstrakciju tekstualnih atributa biblioteke `scikit-learn`.

Uverimo se i praktično u performanse modela tako što ćemo zadati nekoliko paragrafa-rečenica i očitati njihove jezike.  

In [43]:
test_sentences = [
    'This is a language detection test.',
    'Ceci est un test de d\xe9tection de la langue.',
    'Dies ist ein Test, um die Sprache zu erkennen.',
]